In [1]:
##Importing libraries
import langchain
import cv2
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import base64
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
import matplotlib.pyplot as plt
import numpy as np
import sys
from PIL import Image
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from utils import * 
%matplotlib inline

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=2000,
    timeout=None,
    max_retries=2,
)

In [4]:
## Initialize empty dict to store results
final_res_dict = {}

In [5]:
@interact_manual
def run_interactive(selected_images = widgets.SelectMultiple(options=get_file_names('images'),disabled=False,style={'description_width':'initial'},description='Select Images:')):
    global llm
    res_dict = genrate_caption(
                            llm=llm,
                            system_prompt_file='system_prompt.txt',
                            image_list=selected_images)
    final_res_dict.update(res_dict)
    save_output(final_result_dict=final_res_dict)

interactive(children=(SelectMultiple(description='Select Images:', options=('40522491730_088ca7a49c_o.jpg', 'D…

In [6]:
final_res_dict

{'DSC03764.jpg': 'Beneath the stars, he stood in awe, as city lights danced like dreams in the night, a moment of peace amidst the hum of life.',
 'DSC04165.jpg': "On the mountain's edge, he gazed afar, where twinkling lights whispered tales of hope, igniting his spirit with the promise of tomorrow."}

In [7]:
regenerate_prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "{system_prompt}"),
    ("user",
     [
         {
             "type":"text",
             "text":"Image file name: {image_file_name}.You generated the following output for the given image.\n {initial_output} \n. Regenerate a different and better output following your system instruction for the image given below."
             
         },
         {
                    "type": "image_url",
                    "image_url": {"url": "data:image/jpeg;base64,{base64_image}"},
          }
      ]
    )
])

parser = JsonOutputParser()

regenerate_chain = regenerate_prompt|llm|parser

In [ ]:
with open('final_output.json','r') as file:
    final_res_dict = json.load(file)
final_res_dict

In [ ]:
@interact_manual
def f(d = widgets.SelectMultiple(options=final_res_dict.keys(),disabled=False,style={'description_width':'initial'},description='Select Image file:')):
    regen_dict = {k:v for (k,v) in final_res_dict.items() if k in d}
    regen_results = {}
    for k,v in regen_dict.items():
        res_dict = regenrate_caption(chain=regenerate_chain,
                                     system_prompt=generate_sytem_prompt(sys_prompt_file='system_prompt.txt'),
                                     image_file_name=k,
                                     initial_output=v)
        regen_results.update(res_dict)
    final_res_dict.update(regen_results)
    save_output(final_result_dict=final_res_dict)
    display_results(regen_results)

interactive(children=(SelectMultiple(description='Select Image file:', options=('DSC03764.jpg', 'DSC04165.jpg'…

In [9]:
final_res_dict

{'DSC03764.jpg': 'Beneath the stars, he stood in awe, as city lights danced like dreams in the night, a moment of peace amidst the hum of life.',
 'DSC04165.jpg': {'DSC04165.jpg': 'Beneath the vast, starry sky, he stood in silence, where dreams danced like distant lights, weaving a tapestry of hope and wonder in the night.'}}